## Preprocessing du Dataset Heart et Modelisation

In [94]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score,classification_report
import matplotlib.pyplot  as plt
import seaborn as sns
import numpy as np
sns.set()

In [47]:
#Chargement du Dataset
df = pd.read_excel('../dataset/Heart.xlsx')

In [48]:
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,40,homme,AA,140,289,0,Normal,172,Non,0.0,Ascendant,0
1,49,femme,DNA,160,180,0,Normal,156,Non,1.0,Plat,1
2,37,homme,AA,130,283,0,ST,98,Non,0.0,Ascendant,0
3,48,femme,ASY,138,214,0,Normal,108,Oui,1.5,Plat,1
4,54,homme,DNA,150,195,0,Normal,122,Non,0.0,Ascendant,0


In [49]:
df["PENTE"].value_counts()

Plat          460
Ascendant     395
Descendant     63
Name: PENTE, dtype: int64

### Encodage des variables Qualitatives

In [50]:
#Encodage
df['SEXE'] = df['SEXE'].map({"homme":1,"femme":0})
df['TDT'] = df['TDT'].map({'ASY':0,'DNA':1,'AA':2,'AT':3})
df['ECG'] = df['ECG'].map({'Normal':0,'LVH':1,'ST':2})
df['ANGINE'] = df['ANGINE'].map({'Oui':0,'Non':1})
df['PENTE'] = df['PENTE'].map({'Plat':0,'Ascendant':1,'Descendant':2})
df.rename({'DEPRESSION ':'DEPRESSION'},inplace=True)

In [51]:
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,40,1,2,140,289,0,0,172,1,0.0,1,0
1,49,0,1,160,180,0,0,156,1,1.0,0,1
2,37,1,2,130,283,0,2,98,1,0.0,1,0
3,48,0,0,138,214,0,0,108,0,1.5,0,1
4,54,1,1,150,195,0,0,122,1,0.0,1,0


### Normalisation des Variables quantitatives

In [52]:
quant_cols = ['AGE','PAR','CHOLESTEROL','FCMAX','DEPRESSION ']

In [53]:
normalizer = StandardScaler()
df[quant_cols] = normalizer.fit_transform(df[quant_cols].values)

In [62]:
y = df['CŒUR']
data = df.drop('CŒUR',axis=1)

In [65]:
x_train,x_test,y_train,y_test = train_test_split(data,y,test_size=0.2,random_state=10)

### Modelisation

- Modele de Regression Logistique

In [68]:
#Instanciation du modele
ml = LogisticRegression()

In [70]:
#Entrainement du modele
ml.fit(x_train,y_train)

LogisticRegression()

In [72]:
#Performance du modele sur le jeu de donné d'entrainement
train_score = ml.score(x_train,y_train)
train_score

0.8678474114441417

In [74]:
#Performance du modele sur le jeu de donnée de test
test_score = ml.score(x_test,y_test)
test_score

0.842391304347826

In [84]:
#Prediction du model
y_pred = ml.predict(x_test)

In [87]:
#Visualisation du rapport de Classification
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.81      0.83        86
           1       0.84      0.87      0.85        98

    accuracy                           0.84       184
   macro avg       0.84      0.84      0.84       184
weighted avg       0.84      0.84      0.84       184



 - Modele des KMEANS

In [93]:
#Recherche du meilleur paramètre pour le modèle de KNeams
for i in range(1,10):
    kmeans = KNeighborsClassifier(n_neighbors=i)
    kmeans.fit(x_train,y_train)
    print(f'{i} : TEST_SCORE ==>{kmeans.score(x_train,y_train)} | TRAIN_SCORE ==>{kmeans.score(x_test,y_test)}')

1 : TEST_SCORE ==>1.0 | TRAIN_SCORE ==>0.7663043478260869
2 : TEST_SCORE ==>0.9046321525885559 | TRAIN_SCORE ==>0.7445652173913043
3 : TEST_SCORE ==>0.896457765667575 | TRAIN_SCORE ==>0.8260869565217391
4 : TEST_SCORE ==>0.8841961852861036 | TRAIN_SCORE ==>0.8043478260869565
5 : TEST_SCORE ==>0.888283378746594 | TRAIN_SCORE ==>0.8152173913043478
6 : TEST_SCORE ==>0.8787465940054496 | TRAIN_SCORE ==>0.8369565217391305
7 : TEST_SCORE ==>0.8841961852861036 | TRAIN_SCORE ==>0.8369565217391305
8 : TEST_SCORE ==>0.8760217983651226 | TRAIN_SCORE ==>0.8369565217391305
9 : TEST_SCORE ==>0.8773841961852861 | TRAIN_SCORE ==>0.8315217391304348


In [ ]:
km = KNeighborsClassifier(n_neighbors=5)